# Installing neccessary packages


# Install your packages here

In [1]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘reprex’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”


# Load the libraries here

In [1]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ purrr::flatten()        masks jsonlite::flatten()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# trimStringArray function

In [2]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# ternaryOperatorFunction | Vectorized function

In [3]:
ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# Create a vectorized version of strsplit()

In [4]:
strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# Scrape Teilnehmer von WM

In [5]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data
      
      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [6]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [7]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

[1] "Column 1 is not a valid value"


Club,Squad,&oslash-Age,WC particip.,Foreigners,Market Value,&oslash-Market Value,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,28,25.6,16,10.7 %,€1.34bn,€47.71m,england,3299
Brazil,26,27.2,22,88.5 %,€1.05bn,€40.54m,brasilien,3439
Portugal,25,27.3,8,84.0 %,€938.00m,€37.52m,portugal,3300
France,23,26.9,16,78.3 %,€909.50m,€39.54m,frankreich,3377
Germany,24,26.6,20,29.2 %,€818.00m,€34.08m,deutschland,3262
Spain,25,25.9,16,28.0 %,€790.00m,€31.60m,spanien,3375
Argentina,28,27.7,18,96.4 %,€671.00m,€23.96m,argentinien,3437
Belgium,30,27.4,14,76.7 %,€564.50m,€18.82m,belgien,3382
Netherlands,24,26.7,11,50.0 %,€536.95m,€22.37m,niederlande,3379


In [8]:
teilnehmer = df_wm$Club
teilnehmer

[1] "England"       "Brazil"        "Portugal"      "France"       
 [5] "Germany"       "Spain"         "Argentina"     "Belgium"      
 [9] "Netherlands"   "Uruguay"       "Croatia"       "Serbia"       
[13] "Denmark"       "Senegal"       "Poland"        "United States"
[17] "Morocco"       "Switzerland"   "Ghana"         "Mexico"       
[21] "Canada"        "Cameroon"      "Wales"         "South Korea"  
[25] "Japan"         "Ecuador"       "Iran"          "Tunisia"      
[29] "Australia"     "Saudi Arabia"  "Qatar"         "Costa Rica"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [9]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.com/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [10]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(Club == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            #'world cup [0-9]+',
                            #'international friendlies'
                            #'european qualifiers',
                            #'world cup qualification europe'
                            #'euro [0-9]+'
                            #'copa'
                            'confederations cup [0-9]+'
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [12]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [13]:
complete_df = map_df(teilnehmer, scrapeSomething)

[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2014"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2015"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2016"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2017"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2018"
[1] 200
[1] "https://www.transfermarkt.com/Engla

In [14]:
complete_df

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group A,"Sat Jun 15, 2013",9:00 PM,H,,Japan,4-3-3 Attacking,67.423,3:0,Confederations Cup 2013,Brazil
Group A,"Wed Jun 19, 2013",9:00 PM,H,,Mexico,4-3-3 Attacking,50.791,2:0,Confederations Cup 2013,Brazil
Group A,"Sat Jun 22, 2013",9:00 PM,A,,Italy,4-2-3-1,48.874,2:4,Confederations Cup 2013,Brazil
Semi-Finals,"Wed Jun 26, 2013",9:00 PM,H,,Uruguay,4-2-3-1,55.000,2:1,Confederations Cup 2013,Brazil
Final,"Mon Jul 1, 2013",12:00 AM,H,,Spain,4-3-3 Attacking,73.531,3:0,Confederations Cup 2013,Brazil
Group A,"Sun Jun 18, 2017",5:00 PM,H,,Mexico,4-1-3-2,34.372,2:2,Confederations Cup 2017,Portugal
Group A,"Wed Jun 21, 2017",5:00 PM,A,,Russia,4-1-3-2,42.759,0:1,Confederations Cup 2017,Portugal
Group A,"Sat Jun 24, 2017",5:00 PM,A,,New Zealand,4-1-3-2,56.290,0:4,Confederations Cup 2017,Portugal
Semi-Finals,"Wed Jun 28, 2017",8:00 PM,H,,Chile,4-1-3-2,40.855,0:3 on pens,Confederations Cup 2017,Portugal


# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [15]:
#complete_df
write.csv2(complete_df, file='./football_confederations.csv', row.names = F)

## Modelling - Data Cleaning and Transformation

In [16]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football_en.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")


In [17]:
fifa_ranking %>%
    separate(Rang, c('Rang', 'Sym')) %>%
    select(-'Sym') %>%
    mutate(fifa_date = ymd(Datum)) %>%
    select(-c('Datum')) -> fifa_ranking1
fifa_ranking1

date_list = unique(fifa_ranking1$fifa_date)
date_list

Land,Rang,Punkte,fifa_date
<chr>,<chr>,<chr>,<date>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


[1] "2009-09-02" "2009-10-16" "2009-11-20" "2009-12-16" "2010-02-03"
  [6] "2010-03-03" "2010-03-31" "2010-04-28" "2010-05-26" "2010-07-14"
 [11] "2010-08-11" "2010-09-24" "2010-10-20" "2010-11-17" "2010-12-15"
 [16] "2011-01-12" "2011-02-02" "2011-03-09" "2011-04-13" "2011-05-18"
 [21] "2011-06-29" "2011-07-27" "2011-08-24" "2011-09-21" "2011-10-19"
 [26] "2011-11-23" "2011-12-21" "2012-01-18" "2012-02-15" "2012-03-07"
 [31] "2012-04-11" "2012-05-09" "2012-06-06" "2012-07-04" "2012-08-08"
 [36] "2012-09-05" "2012-10-03" "2012-11-07" "2012-12-19" "2013-01-17"
 [41] "2013-02-14" "2013-03-14" "2013-04-11" "2013-05-09" "2013-06-06"
 [46] "2013-07-04" "2013-08-08" "2013-09-12" "2013-10-17" "2013-11-28"
 [51] "2013-12-19" "2014-01-16" "2014-02-13" "2014-03-13" "2014-04-10"
 [56] "2014-05-08" "2014-07-17" "2014-08-14" "2014-09-18" "2014-10-23"
 [61] "2014-11-27" "2014-12-18" "2015-02-12" "2015-03-12" "2015-04-09"
 [66] "2015-05-07" "2015-06-04" "2015-07-09" "2015-08-06" "2015-09-03"
 [71] "2015-10-01" "2015-11-05" "2015-12-03" "2016-01-07" "2016-02-04"
 [76] "2016-03-03" "2016-04-07" "2016-05-05" "2016-05-20" "2016-06-02"
 [81] "2016-07-14" "2016-08-11" "2016-09-15" "2016-10-20" "2016-11-24"
 [86] "2016-12-22" "2017-01-12" "2017-02-09" "2017-03-09" "2017-04-06"
 [91] "2017-05-04" "2017-06-01" "2017-07-06" "2017-08-10" "2017-09-14"
 [96] "2017-10-16" "2017-11-23" "2017-12-21" "2018-01-18" "2018-02-15"
[101] "2018-03-15" "2018-04-12" "2018-05-17" "2018-06-07" "2018-08-16"
[106] "2018-09-20" "2018-10-25" "2018-11-29" "2018-12-20" "2019-02-08"
[111] "2019-04-04" "2019-07-25" "2019-09-10" "2019-09-11" "2019-10-24"
[116] "2019-11-28" "2020-02-20" "2020-03-24" "2020-07-23" "2020-09-17"
[121] "2020-10-22" "2020-11-27" "2020-12-10" "2021-02-03" "2021-02-18"
[126] "2021-06-15" "2021-08-12" "2021-10-20" "2021-10-21" "2021-11-19"
[131] "2021-12-23" "2022-02-10" "2022-03-31" "2022-06-06" "2022-06-23"

In [18]:
data_set

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group C,"Sat Jun 12, 2010",8:30 PM,H,NA,United States,4-4-2 double 6,38.646,1:1,World Cup 2010,England
Group C,"Fri Jun 18, 2010",8:30 PM,H,NA,Algeria,4-4-2 double 6,64.100,0:0,World Cup 2010,England
Group C,"Wed Jun 23, 2010",4:00 PM,A,NA,Slovenia,4-4-2 double 6,36.893,0:1,World Cup 2010,England
Round of 16,"Sun Jun 27, 2010",4:00 PM,A,NA,Germany,4-4-2 double 6,40.510,4:1,World Cup 2010,England
NA,"Wed Mar 3, 2010",9:00 PM,H,NA,Egypt,4-4-2 double 6,80.602,3:1,International Friendlies,England
NA,"Mon May 24, 2010",9:00 PM,H,NA,Mexico,4-2-3-1,88.638,3:1,International Friendlies,England
NA,"Sun May 30, 2010",2:15 PM,H,NA,Japan,4-4-2 double 6,15.326,2:1,International Friendlies,England
NA,"Wed Aug 11, 2010",9:00 PM,H,NA,Hungary,4-2-3-1,72.024,2:1,International Friendlies,England
NA,"Wed Nov 17, 2010",9:00 PM,H,NA,France,4-2-3-1,85.495,1:2,International Friendlies,England


In [19]:
data_set_reduced = data_set %>%
    select(c('Date', 'Venue', 'Opponent', 'Result', 'game_name', 'country'))

In [46]:
lubridate::as_date('Jun 12, 2010')
lubridate:as_datetime('Jun 12, 2010')

Warning message:
“All formats failed to parse. No formats found.”


[1] NA

ERROR: Error in eval(expr, envir, enclos): object 'lubridate' not found


## Obtain goals of enemy and country

In [20]:
data_set_reduced %>%
    separate(Result, c('result_left', 'result_right')) %>%
    mutate(goals_country = ternary_v(result_left, result_right, Venue=='H')) %>%
    mutate(goals_enemy = ternary_v(result_left, result_right, Venue=='A')) -> data_set_increased

data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 304, 307, 420, 474, 531, 533, 534, 589, 617, 618, 660, 693, 832, 833, 839, 840, 841, ...].”


Date,Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Sat Jun 12, 2010",H,United States,1,1,World Cup 2010,England,1,1
"Fri Jun 18, 2010",H,Algeria,0,0,World Cup 2010,England,0,0
"Wed Jun 23, 2010",A,Slovenia,0,1,World Cup 2010,England,1,0
"Sun Jun 27, 2010",A,Germany,4,1,World Cup 2010,England,1,4
"Wed Mar 3, 2010",H,Egypt,3,1,International Friendlies,England,3,1
"Mon May 24, 2010",H,Mexico,3,1,International Friendlies,England,3,1
"Sun May 30, 2010",H,Japan,2,1,International Friendlies,England,2,1
"Wed Aug 11, 2010",H,Hungary,2,1,International Friendlies,England,2,1
"Wed Nov 17, 2010",H,France,1,2,International Friendlies,England,1,2


In [21]:
convert_month_to_decimal = function(month) {
    num = 0
    
    for (i in 1:12) {
        if(month == switch(i,
                          'Jan',
                          'Feb',
                          'Mar',
                          'Apr',
                          'May',
                          'Jun',
                          'Jul',
                          'Aug',
                          'Sep',
                          'Oct',
                          'Nov',
                          'Dec')
          ) {
            num = i
            break;
        }
    }
    return(i)
}

convert_month_to_decimal_v = Vectorize(convert_month_to_decimal)

In [22]:
data_set_increased %>%
    separate(Date, c('weekday', 'month', 'day', 'year'), sep=' ', remove=F) %>%
    separate(day, c('day')) %>%
    mutate(month = convert_month_to_decimal_v(month)) %>%
    mutate(date_of_play = ymd(paste0(year, '-', month, '-', day))) %>%
    select(-c('Date', 'weekday', 'month', 'day', 'year')) -> data_set_cleaned1

data_set_cleaned1

Warning message:
“Expected 1 pieces. Additional pieces discarded in 2399 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17


In [23]:
test_date = function(date_param) {
    for(i in 1:length(date_list)) {
        if(date_list[i] > date_param) {
            break;
        }
    }
    if(i != 1) {
        return(date_list[i-1])
    } else {
        return(date_list[1])
    }
}

test_date_v = Vectorize(test_date)

In [87]:
test = test_date(ymd('2013-06-02'))
test

[1] "2013-05-09"

In [24]:
data_set_cleaned1 %>%
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> data_set_fifa_date

In [25]:
data_set_fifa_date

Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play,fifa_date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12,2010-05-26
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18,2010-05-26
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23,2010-05-26
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27,2010-05-26
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24,2010-04-28
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30,2010-05-26
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17,2010-11-17


## join both data frames using fifa_date

In [96]:
merge(data_set_fifa_date, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) -> newest_df

In [158]:
getFifaRanking = function(country, date) {
    fifa_ranking1 %>%
        filter(Land == country) %>%
        filter(fifa_date == date) %>%
        pluck(2) -> temp
    #print(as.numeric(unlist(temp)))
    
    as.numeric(temp)
}

getFifaRanking_v = Vectorize(getFifaRanking)

In [155]:
#fifa_ranking1 %>%
#    filter(Land == 'Costa Rica') %>%
#    filter(fifa_date == '2009-12-16')
getFifaRanking('Germany', '2022-06-06')

[1] 12


[1] 12

In [159]:
newest_df %>% 
    mutate(rank_opponent = getFifaRanking_v(Opponent, fifa_date)) -> somethingDf

In [161]:
somethingDf %>%
    mutate(rank_opponent = rank_opponent[[1]])

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang,Punkte,rank_opponent
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8,1085,44
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8,1082,44
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9,1087,44
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7,1084,44
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7,1084,44
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7,1076,44
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7,1076,44
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7,1076,44
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7,1076,44


In [74]:
newest_df %>%
    select(-c('Punkte')) -> final_data_set

unique(final_data_set$country)
final_data_set

[1] "Argentina"     "Australia"     "Belgium"       "Brazil"       
 [5] "Cameroon"      "Canada"        "Costa Rica"    "Croatia"      
 [9] "Denmark"       "Ecuador"       "England"       "France"       
[13] "Germany"       "Ghana"         "Iran"          "Japan"        
[17] "Mexico"        "Morocco"       "Netherlands"   "Poland"       
[21] "Portugal"      "Qatar"         "Saudi Arabia"  "Senegal"      
[25] "Serbia"        "South Korea"   "Spain"         "Switzerland"  
[29] "Tunisia"       "United States" "Uruguay"       "Wales"

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7


In [78]:
final_data_set %>%
    filter(result_left == "") %>%
    filter(game_name == 'World Cup 2022') -> world_cup_2022

final_data_set %>%
    filter(result_left != "") -> data_set_without_wc2022

data_set_without_wc2022 %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    mutate(Rank = as.numeric(Rang)) %>%
    select('country', 'Rank', 'goals_country') -> data_set_without_wc2022_cleaned

#test = data_set_without_wc2022_cleaned

#unique(test$country)

#test %>%
#    filter(country == 'United States') -> histogramm

#russia = histogramm$goals_country

#hist(russia)

In [76]:
data_set_without_wc2022 %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    select('country', 'goals_country') -> test

unique(test$country)

test %>%
    filter(country == 'Brazil') -> histogramm

country = histogramm$goals_country

mean(country)

var(country)



[1] "Argentina"     "Australia"     "Belgium"       "Brazil"       
 [5] "Cameroon"      "Canada"        "Costa Rica"    "Croatia"      
 [9] "Denmark"       "Ecuador"       "England"       "France"       
[13] "Germany"       "Ghana"         "Iran"          "Japan"        
[17] "Mexico"        "Morocco"       "Netherlands"   "Poland"       
[21] "Portugal"      "Qatar"         "Saudi Arabia"  "Senegal"      
[25] "Serbia"        "South Korea"   "Spain"         "Switzerland"  
[29] "Tunisia"       "United States" "Uruguay"       "Wales"

[1] 2.160377

[1] 2.574034

In [79]:
data_set_without_wc2022_cleaned

poisson.model = 
    glm(
        goals_country ~ Rank,
        data_set_without_wc2022_cleaned,
        family = poisson(link = 'log')
       )
summary(poisson.model)

country,Rank,goals_country
<chr>,<dbl>,<dbl>
Argentina,8,3
Argentina,8,2
Argentina,9,1
Argentina,7,4
Argentina,7,5
Argentina,7,1
Argentina,7,4
Argentina,7,2
Argentina,7,3



Call:
glm(formula = goals_country ~ Rank, family = poisson(link = "log"), 
    data = data_set_without_wc2022_cleaned)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9164  -1.4119  -0.3244   0.5225   4.4681  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.6133080  0.0251598  24.377  < 2e-16 ***
Rank        -0.0055552  0.0007004  -7.931 2.17e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3252.8  on 2246  degrees of freedom
Residual deviance: 3186.3  on 2245  degrees of freedom
AIC: 7428.3

Number of Fisher Scoring iterations: 5


## Write function that maps a fifa ranking to each date of data_set_cleaned1

In [ ]:
data_set_cleaned1

## Combine fifa ranking and data_set_increased


In [5]:
#bind_cols(data_set_increased, fifa_ranking1)

## Modelling 

In [ ]:
?ls.strstr